## **Bloque 0 – Instalación de librerías adicionales**

In [1]:
!pip install holidays requests

## **Bloque 1 – Imports y carga del Excel**

In [ ]:
import pandas as pd
import numpy as np
import requests
import holidays
from datetime import datetime

# Opcional: si vas a usar Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

# Ruta al archivo (ajusta según donde lo tengas)
EXCEL_PATH = "./data/trafico_consolidado.xlsx"  # por ejemplo

# Lectura del Excel
df_raw = pd.read_excel(EXCEL_PATH)

print(df_raw.shape)
df_raw.head()

(40217, 6)


,fecha,total,sentido_2,sentido_1,peaje,archivo
0,2021-12-01,0,0,0,AGUAS NEGRAS,C:\Users\adelgado\Documents\Personal\Universid...
1,2021-12-02,0,0,0,AGUAS NEGRAS,C:\Users\adelgado\Documents\Personal\Universid...
2,2021-12-03,0,0,0,AGUAS NEGRAS,C:\Users\adelgado\Documents\Personal\Universid...
3,2021-12-04,3946,1916,2030,AGUAS NEGRAS,C:\Users\adelgado\Documents\Personal\Universid...
4,2021-12-05,7679,3881,3798,AGUAS NEGRAS,C:\Users\adelgado\Documents\Personal\Universid...


## **Bloque 2 – Limpieza básica, tipos y filtrado de fechas**

In [3]:
df = df_raw.copy()

# 1. Asegurar tipo fecha (normalizada a medianoche)
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce').dt.normalize()

# 2. Filtrar desde 2022-01-01 en adelante (ignora todo 2021)
fecha_min = pd.to_datetime("2022-01-01")
df = df[df['fecha'] >= fecha_min].copy()

# 3. Eliminamos columna 'archivo' (no se necesita)
if 'archivo' in df.columns:
    df = df.drop(columns=['archivo'])

# 4. Verificamos nombres y los reordenamos:
#   En tu archivo están como: fecha, total, sentido_2, sentido_1, peaje
#   Los queremos como: fecha, peaje, sentido_1, sentido_2, total
expected_cols = ['fecha', 'total', 'sentido_2', 'sentido_1', 'peaje']
print("Columnas actuales:", df.columns.tolist())

# Reordenamos y renombramos de forma explícita
df = df[['fecha', 'peaje', 'sentido_1', 'sentido_2', 'total']]

# 5. Función para normalizar enteros (ignorando decimales y símbolos)
def to_int_no_decimals(x):
    """
    Convierte un valor a entero:
    - Si tiene '.' o ',', ignora todo lo que venga después.
    - Elimina cualquier símbolo extraño (como $ o espacios).
    - Si queda vacío o NaN, devuelve 0.
    """
    if pd.isna(x):
        return 0
    s = str(x).strip()
    # Cortar en primer '.' o ','
    for sep in ('.', ','):
        idx = s.find(sep)
        if idx != -1:
            s = s[:idx]
            break
    # Dejar sólo dígitos y posible signo negativo al inicio
    clean = []
    for i, ch in enumerate(s):
        if ch.isdigit():
            clean.append(ch)
        elif ch in ['-','+'] and i == 0:
            clean.append(ch)
        # todo lo demás lo ignoramos
    clean = ''.join(clean)
    if clean in ['', '-', '+']:
        return 0
    try:
        return int(clean)
    except ValueError:
        return 0

for col in ['sentido_1', 'sentido_2', 'total']:
    df[col] = df[col].apply(to_int_no_decimals).astype(int)

df.dtypes

Columnas actuales: ['fecha', 'total', 'sentido_2', 'sentido_1', 'peaje']


,0
fecha,datetime64[ns]
peaje,object
sentido_1,int64
sentido_2,int64
total,int64


## **Bloque 3 – Agrupar por fecha y peaje (sumar duplicados) y ordenar**

In [4]:
# Agrupar por fecha y peaje, sumando las columnas numéricas
df_grouped = (
    df
    .groupby(['fecha', 'peaje'], as_index=False)[['sentido_1', 'sentido_2', 'total']]
    .sum()
)

# Ordenar por fecha (más antigua a más reciente) y por peaje
df_grouped = df_grouped.sort_values(['fecha', 'peaje']).reset_index(drop=True)

df_grouped.head(20)

,fecha,peaje,sentido_1,sentido_2,total
0,2022-01-01,AGUAS NEGRAS,2810,1786,4596
1,2022-01-01,CASABLANCA,4248,3597,7845
2,2022-01-01,COCORNA (EL SANTUARIO),3034,2615,5649
3,2022-01-01,CURITÍ,1988,2028,4016
4,2022-01-01,LA GOMEZ,2342,1419,3761
5,2022-01-01,LA PARADA,0,7773,7773
6,2022-01-01,LOBOGUERRERO,1036,712,1748
7,2022-01-01,LOS CUROS,1766,2058,3824
8,2022-01-01,MORRISON,3075,2166,5241
9,2022-01-01,OIBA,1779,1494,3273


## **Bloque 4 – Variable exógena: Clasificación de tipo de día (entre semana / fin de semana / festivo)**

In [5]:
# Determinar rango de años presente en los datos
years = sorted(df_grouped['fecha'].dt.year.unique())
print("Años en los datos:", years)

# Calendario de festivos de Colombia
co_holidays = holidays.country_holidays('CO', years=years)

def clasificar_dia(fecha):
    """
    Devuelve:
    - 'festivo' si es festivo en Colombia
    - 'fin_de_semana' si es sábado o domingo
    - 'entre_semana' en caso contrario
    """
    # fecha es Timestamp; usamos .date() para compararlo con el set de festivos
    d = fecha.date()
    if d in co_holidays:
        return 'festivo'
    elif fecha.weekday() >= 5:  # 5=sábado, 6=domingo
        return 'fin_de_semana'
    else:
        return 'entre_semana'

df_grouped['tipo_dia'] = df_grouped['fecha'].apply(clasificar_dia)

df_grouped[['fecha', 'peaje', 'tipo_dia']].head(20)

Años en los datos: [np.int32(2022), np.int32(2023), np.int32(2024), np.int32(2025)]


,fecha,peaje,tipo_dia
0,2022-01-01,AGUAS NEGRAS,festivo
1,2022-01-01,CASABLANCA,festivo
2,2022-01-01,COCORNA (EL SANTUARIO),festivo
3,2022-01-01,CURITÍ,festivo
4,2022-01-01,LA GOMEZ,festivo
5,2022-01-01,LA PARADA,festivo
6,2022-01-01,LOBOGUERRERO,festivo
7,2022-01-01,LOS CUROS,festivo
8,2022-01-01,MORRISON,festivo
9,2022-01-01,OIBA,festivo


## **Bloque 5 – Variable exógena: TRM (precio del dólar) diaria vía API**

In [6]:
# Rango de fechas que necesitamos
start_date = df_grouped['fecha'].min().strftime("%Y-%m-%d")
end_date   = df_grouped['fecha'].max().strftime("%Y-%m-%d")
print("Rango de fechas TRM:", start_date, "a", end_date)

TRM_ENDPOINT = "https://www.datos.gov.co/resource/32sa-8pi3.json"

params = {
    # Campos que queremos: vigenciadesde (fecha) y valor (TRM)
    "$select": "vigenciadesde,valor",
    "$where": f"vigenciadesde between '{start_date}' and '{end_date}'",
    "$order": "vigenciadesde ASC",
    "$limit": 5000  # suficiente para varios años de datos diarios
}

response = requests.get(TRM_ENDPOINT, params=params)
response.raise_for_status()
trm_json = response.json()

trm_df = pd.DataFrame(trm_json)
if not trm_df.empty:
    trm_df['fecha'] = pd.to_datetime(trm_df['vigenciadesde']).dt.normalize()
    trm_df['trm_dolar'] = trm_df['valor'].astype(float)
    trm_df = trm_df[['fecha', 'trm_dolar']]
else:
    # Si la consulta no trae nada (por problema de API), creamos un df vacío
    trm_df = pd.DataFrame(columns=['fecha', 'trm_dolar'])

trm_df.head()

Rango de fechas TRM: 2022-01-01 a 2025-10-01


,fecha,trm_dolar
0,2022-01-04,4082.75
1,2022-01-05,4084.11
2,2022-01-06,4042.36
3,2022-01-07,4039.31
4,2022-01-08,4043.46


In [7]:
# Merge de la TRM con el DataFrame principal
df_merged = df_grouped.merge(trm_df, on='fecha', how='left')

df_merged[['fecha', 'trm_dolar']].head(20)

,fecha,trm_dolar
0,2022-01-01,NaN
1,2022-01-01,NaN
2,2022-01-01,NaN
3,2022-01-01,NaN
4,2022-01-01,NaN
5,2022-01-01,NaN
6,2022-01-01,NaN
7,2022-01-01,NaN
8,2022-01-01,NaN
9,2022-01-01,NaN


In [8]:
# Ordenamos por fecha para permitir el relleno correcto
df_merged = df_merged.sort_values("fecha")

# Relleno hacia adelante: la TRM del último día hábil se extiende a fines de semana/festivos
df_merged['trm_dolar'] = df_merged['trm_dolar'].astype(float)
df_merged['trm_dolar'] = df_merged['trm_dolar'].ffill()

# Si quieres también rellenar los primeros días del dataset
df_merged['trm_dolar'] = df_merged['trm_dolar'].bfill()

df_merged[['fecha', 'trm_dolar']].head(20)

,fecha,trm_dolar
0,2022-01-01,4082.75
17,2022-01-01,4082.75
16,2022-01-01,4082.75
15,2022-01-01,4082.75
14,2022-01-01,4082.75
12,2022-01-01,4082.75
11,2022-01-01,4082.75
10,2022-01-01,4082.75
9,2022-01-01,4082.75
13,2022-01-01,4082.75


## **Bloque 6 – Exportar a CSV final**

In [ ]:
OUTPUT_CSV = "./data/trafico_limpio.csv"

df_final = df_merged.copy()

# Exportar con fecha sin hora (formato YYYY-MM-DD)
df_final.to_csv(OUTPUT_CSV, index=False, date_format="%Y-%m-%d")

print("Archivo guardado en:", OUTPUT_CSV)
df_final.head()

Archivo guardado en: /content/trafico_limpio.csv


,fecha,peaje,sentido_1,sentido_2,total,tipo_dia,trm_dolar
0,2022-01-01,AGUAS NEGRAS,2810,1786,4596,festivo,4082.75
17,2022-01-01,ZAMBITO,2561,1429,3990,festivo,4082.75
16,2022-01-01,SAN JUAN,937,1169,2106,festivo,4082.75
15,2022-01-01,SAN DIEGO,886,901,1787,festivo,4082.75
14,2022-01-01,SABOYA,2181,1375,3556,festivo,4082.75
